In [1]:
import pandas as pd

## Stacking & Unstacking

Exploration of pivoting / unpivoting using the inbuilt `.stack()` & `.unstack()` functionality. 

[Reference Video](https://www.youtube.com/watch?v=kJsiiPK5sxs)

In [ ]:
###
# Code for re-labeling multi indexed columns, to avoid having to use index tuples
###

# tips_us_copy = tips_us.copy()

# tips_us_copy.columns = ['__'.join(col).strip() for col in tips_us.columns.values]